(onboardone)=
# Import a World Bank Eviews model from its solution file (.wf1)
This notebook takes a .wf1 workfile and transforms it into a modelflow model.

Most standard World Bank models should work with limited intervention.  Some using unusual techniques or variable definitions may require intervention by the user.

## Overview of the import process

The overall import process is performed by a special a ModelFlow class named:**GrabWfModel**. Certain steps require the use of EViews itself.  This howoto was designed usinng EViews version 12, but and has been tested with versions 13 and 14.

Steps to follow:
 1. Start EViews and open the solution file 
    -    If returning to this step after an initial error perform transformations on the data (needed in some cases where special EViews functions are used in the model).
    -    The UNlink the model. This transforms linked equations into explicit equations in the EViews model object. 
    -    Save the revised model and data as a .wf2 file. (the .wf2 is a JSON format that can be easily read into python).
    **QCVD?  Same name with ```_modelflow appended```**
 5. Close Eviews.
 6. The wf2 file is read as a json file. 
 7. Relevant objects are extracted. 
 7. The MFMSA variable is  extracted, to be saved in the dumpfile. 
 8. The equations are transformed and normalized to modelflow format and classified into identities and stochastic
 9. Stochastic equations are enriched by add_factor and fixing terms (dummy + fixing value)  
 9. For Stochastic equations new fitted variables are generated - without add add_factors and dummies.  
 9. A model to generate fitted variables is created  
 9. A model to generate add_factors is created. 
 9. A model encompassing the original equations, the model for fitted variables and for add_factors is created. 
 9. The data series and scalars are shoveled into a Pandas dataframe 
     - Some special series are generated as the expression can not be incorporated into modelflow model specifications
     - The model for fitted values is simulated in the specified timespan
     - The model for add_factors is simulated in the timespan set in MFMSA
 10. The data descriptions are extracted into a dictionary. 
    - Data descriptions for dummies, fixed values, fitted values and add_factors are derived. 
 11. Now we have a model and a dataframe with all variables which are needed.
 
The GrapWfModel instance in general keeps most of the steps so the developer can inspect the the different steps.  


## Prerequisites  

The import process requires EViews and tehrefore cannot be run on a non-windows computerthat does not have EViews 12 or later installed.  It also assumes that the python library `pyeviews` has been installed (`conda install pyeviews`).  `pyeviews` uses the EViews com interface to control eviews, allowing a python script to directly manipulate parts of the eviews model using eviews itself.  This is required only for the import process.


In [1]:
%matplotlib Notebook #tell matplotlib we are running in a notebook

In [2]:
from pathlib import Path

from modelclass import model 
from modelgrabwf2 import GrabWfModel  #this is the class (part of the Modelflow Library) that will translate
                                      # the ewbvis model and equations into modelflow business logic and
                                      # import the data into pandas
model.widescreen() # Makes modelflow aware we may be running under a wider than normal screem
model.scroll_off() # tells modelflow not to scroll outputs

<IPython.core.display.Javascript object>

In [3]:
 %load_ext autoreload
 %autoreload 2

## Model specific transformations 

Some expressions found in World Bank equations are functions that have no direct equvalent in python. `GrabWfModel()` is aware of many of these and peforms he necessary adjustments automatically. Others it may not be aware of.  

In cases where manual adjustments are necessary, the `GrabWfModel()`procedure will error out and the equatiopns will have to be re-written perhaps as two or more equations manually. 

We perform these changes in eviews using the `pyeviews` function by entering eviews command and subnitting them to the `GrabWfModel()`function. 

For example, and eviews equation may include a cross equation constraint by specifying a parameter in one equation to be equal to the value of a parameter in another equation. 

In an eviews model this might be written as 

`equation _mycrossequation.ls delog(vary) = c(1) + SomeOtherEquation.coeff(2)*Somevar`

which would instruct eviews to run a least squared regression estimating only the intercept `c(1)` and imposting as the slope variable the second coefficient from the second equation `SomeOtherEquation.coeff(2)`.  Modelflow does not have this dynamic mecanism to refer to coefficients in other equations so we must rewrite the equation by first assigning the value of Secondequation.coeff(2) to a scalar and the using that scalar in the equation instead.

We would do this by executing (in EViews) the following  line.

```
'Eviews code that assigns to a scalar the value of the coeeficint
Scalar _SomeOtherEquation_at_COEF_2 = SomeOtherEquation.@COEF(+2)
```
We will then want to rewrite the equation (in modelflow) substituting the expression 
`SomeOtherEquation.@coeff(2)` with the `scalar_SomeOtherEquation_at_COEF_2`.

This is accomplished by passing to `GrabWFModel()` two parameters, the first that contains the ewviws code to be run and the second the change to the equation we need.  These are then run by `GrabWFModel()` before generating the Modelflow version of the model.

Below is a real-life example requried when importing a model for Moldova.

```
mda_eviews_run_lines = ['Scalar _MDASBBREV_at_COEF_2 = _MDASBBREV.@COEF(+2)']
mda_trans = lambda input: input.replace('_MDAsbbrev.@coef(2)','_MDASBBREV_at_COEF_2')  
```

In this second example, the original eviews equation included an expression that calculated the mean value of a variable on the fly and used it in an equation.  Here we use two EViews commands `smpl @all` and `series MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD = @MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")` to create a variable that contains teh value the expression would generate and inserts that into the equation instead of the expression.

```
# Another example 
ago_eviews_run_lines = ['smpl @ALL','series MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD = @MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")']
ago_trans = lambda  input : input.replace('@MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")','MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD') 

```
|The GrabWBModel parameters| Purpose|
|:--|:--|
|**eviews_run_lines**|Specifies a comma-delimited list of Eviews command lines to be run. Typically these generate a variable to stand in for an inline function that appears in an equation in the model object. There is not limit to the number of Eviews commnds that can be submitted| 
|**country_trans**|Specifies a comma-delimited list of replacements (python code) to be done to the formulas before they are processed further. |


Example use:
```
all_about_mda = GrabWfModel(r'wfs\bolsoln.wf1', 
                  eviews_run_lines= mda_eviews_run_lines,
                  country_trans    =  mda_trans,
                  make_fitted = True,        # make equatios for fitted values of stocastic equations 
                  do_add_factor_calc=True,   # Calculate the add factors which makes the stocastic equations match                     fit_start = 2000,          # Solve start for the model
                  fit_end   = None,          # Solve End date, 
                                             # if None taken it will use the value specified 
                                             # in the EViews MFMSAOptions variable 
                    disable_progress =True   # Supresses the progress bar
                           ) 
```

Not all inline EViews functions (@ functions) will throw errors.  The import routine klnows how to deal with many of them.  The table below outlines those it is aware of and what they do.

|EViews in line function | What it does |
|:--|:--|
|@During(Date,[Date])|Generates a dummy with value one at date (or over a range of dates if more than one date is specified|
|@Recode(variable,Logical condition)|Generates a dummy with value one for those dates where the logical condition is true|



## An example

The below command tries to import a model for Bolivia without any modification.  The import fails, with `GrabWfModel()` reporting:

```
Processing the model:BOL
Probably errors as @ in lines:

Eviews line      :@IDENTITY BOLBFCAF2BOP_  = (1  - @DURING("1980 2021"))  * @between(reserveRatio3  , 1  / 2  , 100000)  * reserveRatio3
Original line     :@IDENTITY BOLBFCAF2BOP_  = (1  - DURING_1980_2021)  * @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)  * RESERVERATIO3
New modelflow line:BOLBFCAF2BOP_=(1-DURING_1980_2021)*@BETWEEN(RESERVERATIO3,1/2,100000)*RESERVERATIO3
```


In [12]:
try:
    all_about_mda = GrabWfModel(r'wfs\bolsoln.wf1', 
                  #eviews_run_lines= mda_eviews_run_lines,
                  #country_trans    =  mda_trans,
                    make_fitted = True,        # make equatios for fitted values of stocastic equations 
                    do_add_factor_calc=True,   # Calculate the add factors which makes the stocastic equations match    
                    fit_start = 2000,          # Start of calculation of fittet model in baseline (to have some historic values) 
                    fit_end   = None,           # end of calc for fittted model, if None taken from mdmfsa options  
                    disable_progress =True     # Better for jupyter book 
                           ) 
except 


Reading C:\mflow\modelflow-manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln.wf1
Assummed model name: BOL
The model: BOL is unlinked 
Writing C:\mflow\modelflow-manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln_modelflow.wf2
Model name: BOL

Processing the model:BOL
Probably errors as @ in lines:

Eviews line      :@IDENTITY BOLBFCAF2BOP_  = (1  - @DURING("1980 2021"))  * @between(reserveRatio3  , 1  / 2  , 100000)  * reserveRatio3
Original line     :@IDENTITY BOLBFCAF2BOP_  = (1  - DURING_1980_2021)  * @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)  * RESERVERATIO3
New modelflow line:BOLBFCAF2BOP_=(1-DURING_1980_2021)*@BETWEEN(RESERVERATIO3,1/2,100000)*RESERVERATIO3


Exception: @ in lines 

## Debugging the error


As we know from the preceding table that the transformation logic in GrabWFModel() is aware of teh @During function it must be the @Between that is causing trouble.

The EViews help tells us:

```
@between(series, val1, val2)  Creates a dummy variable equal to 1 for observations where series is greater than or equal to val1 and less than or equal to val2.
```

We can reproduce the logic by adding a variable with the result of the eviews calcualtion and inserting it into the equation.

`BOL_Eviews='series Reservationratio3_at_Between= @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)'`

and then adding a python commmand to perform the susbtitution in the equation.

`BOL_trans='lambda input: input.replace('@BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)','_Reservationratio3_at_Between')'`



The error indicates that the import routine did not know how to deal with the line

`@IDENTITY BOLBFCAF2BOP_  = (1  - @DURING("1980 2021"))  * @between(reserveRatio3  , 1  / 2  , 100000)  * reserveRatio3`

and that the problem had to do with one of the @ expressions.  



**Problem below**
eviews_run_lines should be a list that is 

```BOL_Eviews=["series _Reservationratio3_at_Between= @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)"]```

The BOL_trans looks ok.

A more simple way would in this case be: 



In [18]:

BOL_Eviews=['smpl @ALL',"series _Reserveratio3_at_Between= @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)"];
BOL_trans=lambda input: input.replace("@BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)","__RESERVERATIO3_AT_BETWEEN")

all_about_mda = GrabWfModel(r'wfs\bolsoln.wf1', 
                  eviews_run_lines= BOL_Eviews,
                  country_trans    =  BOL_trans,
                    make_fitted = True,        # make equations for fitted values of stochastic equations 
                    do_add_factor_calc=True,   # Calculate the add factors which makes the stocastic equations match    
                    fit_start = 2000,          # Start of calculation of fittet model in baseline (to have some historic values) 
                    fit_end   = None,           # end of calc for fittted model, if None taken from mdmfsa options  
                    disable_progress =True     # Better for jupyter book 
                           ) 


Reading C:\mflow\modelflow-manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln.wf1
Assummed model name: BOL
Eviewsline to run :smpl @ALL
Eviewsline to run :series _Reserveratio3_at_Between= @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)
The model: BOL is unlinked 
Writing C:\mflow\modelflow-manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln_modelflow.wf2
Model name: BOL

Processing the model:BOL
Probably errors as @ in lines:

Eviews line      :@IDENTITY BOLBFCAF2BOP_  = (1  - @DURING("1980 2021"))  * @between(reserveRatio3  , 1  / 2  , 100000)  * reserveRatio3
Original line     :@IDENTITY BOLBFCAF2BOP_  = (1  - DURING_1980_2021)  * @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)  * RESERVERATIO3
New modelflow line:BOLBFCAF2BOP_=(1-DURING_1980_2021)*@BETWEEN(RESERVERATIO3,1/2,100000)*RESERVERATIO3


Exception: @ in lines 

## Check if each equation on its own result in the values provided. 
aka: residual check <br> 
If they are not pretty close, something is very wrong. 

In [20]:
all_about_mda.test_model(all_about_mda.start,all_about_mda.end,maxerr=100,tol=1,showall=0)   # tol determins the max acceptable absolute difference 

MDA calculated  

Chekking residuals for MDA 2014 to 2025


## Extract the model and the baseline
**all_about_mda** has a lot of content including. 
- .mmodel is the model instance
- .base_input is the baseline where the add factors and the fitted values are calculated 

In [23]:
mmda    = all_about_mda.mmodel       # the model instance  
baseline = all_about_mda.base_input

## Run the model 
In order to achieve numerical stability Gauss-Seidle has to be dampened: alfa=0.7 makes sure that the solution does not explode. 
The convergence criteria is tightend a lot. 

In [24]:
res = mmda(all_about_mda.base_input,2016,2040,silent=1,alfa=1,ldumpvar=0)
mmda.basedf = all_about_mda.base_input

## And the simulation result is also fine. 
Here the percent difference is displayed

In [9]:
mmda['mdaGGEXPCAPTCN mdaNYGDPMKTPCN mdaGGDBTTOTLCN mdaBNCABFUNDCD']

## Look at a stochastic variable 
Here the equations undergo more phases 

In [10]:
all_about_mda.all_frml_dict['MDABFCAFCAPTCD'].fprint


Endo_var        : MDABFCAFCAPTCD
Original        : MDABFCAFCAPTCD/(MDANYGDPMKTPCN/MDAPANUSATLS)=0.00282736002227067
Preprocessed    : MDABFCAFCAPTCD/(MDANYGDPMKTPCN/MDAPANUSATLS)=0.00282736002227067
Normalized      : MDABFCAFCAPTCD = (MDABFCAFCAPTCD_A*MDANYGDPMKTPCN/MDAPANUSATLS+MDANYGDPMKTPCN* (0.00282736002227067) /MDAPANUSATLS) * (1-MDABFCAFCAPTCD_D)+ MDABFCAFCAPTCD_X*MDABFCAFCAPTCD_D 
Calc_add_factor : MDABFCAFCAPTCD_A = MDABFCAFCAPTCD*MDAPANUSATLS/MDANYGDPMKTPCN- ((0.00282736002227067)) 
Fitted          : MDABFCAFCAPTCD_FITTED = MDANYGDPMKTPCN* (0.00282736002227067) /MDAPANUSATLS
Eviews          : MDABFCAFCAPTCD/(MDANYGDPMKTPCN/MDAPANUSATLS) = 0.00282736002227067


## Look a all the modelflow frmls
Notice after the "original" model the equations for the "fitted" values have been added. <br>
Also in the end of the listing the specification of the model which calculates the add factors if a variable is fixed. When processing the equations the ```model``` class will process this this model separately and create a model instance 
which is used to calculate add factors in case 

In [11]:
print(mmda.equations)

FRML  <> MDAPSTAR = (1/MDANYGDPTFP)**(1/MDANYYWBTOTL_)*(MDANYWRTTOTLXN/MDANYYWBTOTL_)*(MDANYGDPPOTLKN/MDANEGDIKSTKKN)**((1-MDANYYWBTOTL_)/MDANYYWBTOTL_) $
FRML  <> MDAGR = ((log(MDANYGDPTFP))-(log(MDANYGDPTFP(-1))))/MDANYYWBTOTL_+((log(MDASPPOPWORK))-(log(MDASPPOPWORK(-1)))) $
FRML <Z,EXO> MDANYGDPFCSTXN = (MDANYGDPFCSTXN(-1)*EXP(MDANYGDPFCSTXN_A+ (0.197543704108975*((LOG(MDANYGDPFCSTXN(-1)))-(LOG(MDANYGDPFCSTXN(-2))))+(1-0.197543704108975)*(0.3*MDAINFLEXPT/100+(1-0.3)*((LOG(MDAPSTAR))-(LOG(MDAPSTAR(-1)))))+0.3*MDANYGDPGAP_/100+0.0103490916251564*DURING_2011-0.119727211086869*DURING_2009) )) * (1-MDANYGDPFCSTXN_D)+ MDANYGDPFCSTXN_X*MDANYGDPFCSTXN_D  $
FRML <Z,EXO> MDAFMLBLPOLYXN = (100*MDAFMLBLPOLYXN_A+100* (0.7*MDAFMLBLPOLYXN(-1)/100+(1-0.7)*(0.108512280547198+1.5*(((LOG(MDANECONPRVTXN))-(LOG(MDANECONPRVTXN(-1))))-MDAINFLEXPT/100)+0.6*MDANYGDPGAP_/100)) ) * (1-MDAFMLBLPOLYXN_D)+ MDAFMLBLPOLYXN_X*MDAFMLBLPOLYXN_D  $
FRML  <> MDAINTRDXN = MDAFMLBLPOLYXN+MDAINTRDDIFF $
FRML  <> MDAINTREX

In [12]:
with mmda.set_smpl(2020,2023):
    print(mmda.mdaGGEXPCAPTCN.show)                        

Endogeneous: MDAGGEXPCAPTCN: Expenditure: Investment LCU millions
Formular: FRML <Z,EXO> MDAGGEXPCAPTCN = (MDANYGDPMKTPXN*MDAGGEXPCAPTCN(-1)*EXP(MDAGGEXPCAPTCN_A+ (-0.3*(LOG(MDAGGEXPCAPTCN(-1)/MDANYGDPMKTPXN(-1))-LOG(MDANYGDPPOTLKN(-1)))+0.27663427827676*((LOG(MDAGGEXPCAPTCN(-1)/MDANYGDPMKTPXN(-1)))-(LOG(MDAGGEXPCAPTCN(-2)/MDANYGDPMKTPXN(-2))))+(1-0.27663427827676)*MDAGR-1.03766125029565) )/MDANYGDPMKTPXN(-1)) * (1-MDAGGEXPCAPTCN_D)+ MDAGGEXPCAPTCN_X*MDAGGEXPCAPTCN_D  $

MDAGGEXPCAPTCN  : Expenditure: Investment LCU millions
MDAGGEXPCAPTCN_A: Add factor:Expenditure: Investment LCU millions
MDAGGEXPCAPTCN_D: Fix dummy:Expenditure: Investment LCU millions
MDAGGEXPCAPTCN_X: Fix value:Expenditure: Investment LCU millions
MDAGR           : 
MDANYGDPMKTPXN  : 
MDANYGDPPOTLKN  : 

Values :


,2020,2021,2022,2023
Base,"5,506.20","5,497.10","6,604.18","7,989.78"
Last,"5,506.20","5,497.10","6,604.18","7,989.78"
Diff,-0.00,0.00,-0.00,-0.00


Input last run:


,2020,2021,2022,2023
MDAGGEXPCAPTCN(-1),"4,783.20","5,506.20","5,497.10","6,604.18"
MDAGGEXPCAPTCN(-2),"4,517.20","4,783.20","5,506.20","5,497.10"
MDAGGEXPCAPTCN_A,-0.02,-0.16,-0.02,-0.04
MDAGGEXPCAPTCN_D,0.00,0.00,0.00,0.00
MDAGGEXPCAPTCN_X,0.00,0.00,0.00,0.00
MDAGR,0.02,0.02,0.02,0.02
MDANYGDPMKTPXN,1.81,1.89,2.09,2.32
MDANYGDPMKTPXN(-1),1.72,1.81,1.89,2.09
MDANYGDPMKTPXN(-2),1.63,1.72,1.81,1.89
MDANYGDPPOTLKN(-1),"121,339.21","125,515.91","129,531.84","133,500.17"


Input base run:


,2020,2021,2022,2023
MDAGGEXPCAPTCN(-1),"4,783.20","5,506.20","5,497.10","6,604.18"
MDAGGEXPCAPTCN(-2),"4,517.20","4,783.20","5,506.20","5,497.10"
MDAGGEXPCAPTCN_A,-0.02,-0.16,-0.02,-0.04
MDAGGEXPCAPTCN_D,0.00,0.00,0.00,0.00
MDAGGEXPCAPTCN_X,0.00,0.00,0.00,0.00
MDAGR,0.02,0.02,0.02,0.02
MDANYGDPMKTPXN,1.81,1.89,2.09,2.32
MDANYGDPMKTPXN(-1),1.72,1.81,1.89,2.09
MDANYGDPMKTPXN(-2),1.63,1.72,1.81,1.89
MDANYGDPPOTLKN(-1),"121,339.21","125,515.91","129,531.84","133,500.17"


Difference for input variables


,2020,2021,2022,2023
MDAGGEXPCAPTCN(-1),-0.00,-0.00,0.00,-0.00
MDAGGEXPCAPTCN(-2),-0.00,-0.00,-0.00,0.00
MDAGGEXPCAPTCN_A,0.00,0.00,0.00,0.00
MDAGGEXPCAPTCN_D,0.00,0.00,0.00,0.00
MDAGGEXPCAPTCN_X,0.00,0.00,0.00,0.00
MDAGR,0.00,0.00,0.00,0.00
MDANYGDPMKTPXN,-0.00,0.00,-0.00,-0.00
MDANYGDPMKTPXN(-1),-0.00,-0.00,0.00,-0.00
MDANYGDPMKTPXN(-2),-0.00,-0.00,-0.00,0.00
MDANYGDPPOTLKN(-1),-0.00,-0.00,-0.00,-0.00



None


In [25]:
mmda.modeldump('test.pcim')

In [26]:
!dir *.pcim

 Volume in drive C has no label.
 Volume Serial Number is C2DB-095E

 Directory of c:\wb new\Modelflow\working_paper\thebook\content\howto\onboard\eviews

30-06-2022  12:27         1.029.276 test.pcim
               1 File(s)      1.029.276 bytes
               0 Dir(s)  787.101.802.496 bytes free
